In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler


In [3]:
#Datensatz laden
data = pd.read_csv("food.csv")


In [5]:
# Daten inspizieren
print(data.head())
print(data.dtypes)

       Unnamed: 0  RedMeat  WhiteMeat  Eggs  Milk  Fish  Cereals  Starch  \
0         Albania     10.1        1.4   0.5   8.9   0.2     42.3     0.6   
1         Austria      8.9       14.0   4.3  19.9   2.1     28.0     3.6   
2         Belgium     13.5        9.3   4.1  17.5   4.5     26.6     5.7   
3        Bulgaria      7.8        6.0   1.6   8.3   1.2     56.7     1.1   
4  Czechoslovakia      9.7       11.4   2.8  12.5   2.0     34.3     5.0   

   Nuts  Fr.Veg  
0   5.5     1.7  
1   1.3     4.3  
2   2.1     4.0  
3   3.7     4.2  
4   1.1     4.0  
Unnamed: 0     object
RedMeat       float64
WhiteMeat     float64
Eggs          float64
Milk          float64
Fish          float64
Cereals       float64
Starch        float64
Nuts          float64
Fr.Veg        float64
dtype: object


In [6]:
#numerische Werte ausschließen
numeric_data = data.select_dtypes(include=['number'])

In [9]:
from sklearn.preprocessing import StandardScaler

In [10]:
# Z-Transformation auf numerischen Daten
scaler = StandardScaler()
scaled_data = scaler.fit_transform(numeric_data)

In [11]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [12]:
#KMeans clustering
best_score = -1
best_k = 2
best_model = None

In [14]:
# Teste K von 2 bis 5
for k in range(2, 6):
    model = KMeans(n_clusters=k, random_state=42)
    labels = model.fit_predict(scaled_data)
    score = silhouette_score(scaled_data, labels)
    print(f"K={k}, Silhouette Score={score:.4f}")
    
    if score > best_score:
        best_score = score
        best_k = k
        best_model = model

print(f"Best k: {best_k}")

K=2, Silhouette Score=0.3271
K=3, Silhouette Score=0.3352
K=4, Silhouette Score=0.2799
K=5, Silhouette Score=0.2140
Best k: 3
